# Uso del modelo de ML

Este notebook es una continuación de 'PI02_01_Procesos.ipynb'.

Ahora aplicaremos todos los procesos que ya aplicamos a *hospitalizaciones_train.csv* sobre *hospitalizaciones_test.csv*.

No detallaremos demasiado, ya lo hicimos en el primer notebook.

In [6]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [2]:
df_test = pd.read_csv('.\Datasets\hospitalizaciones_test.csv')

In [3]:
df_test = df_test[['Department', 'staff_available', 'Age', 'gender', 'Type of Admission', 'Severity of Illness', 'health_conditions']]

In [4]:
df_test.drop(columns=['health_conditions'], inplace=True)

In [7]:
edad_cat = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
complejidad_cat = ['Minor', 'Moderate', 'Extreme']

ordinal_encoder = OrdinalEncoder(categories=[edad_cat, complejidad_cat])
df_test[['Age', 'Severity of Illness']] = ordinal_encoder.fit_transform(df_test[['Age', 'Severity of Illness']])

In [8]:
labels = ['Department', 'gender', 'Type of Admission']

ohe = OneHotEncoder(categories='auto')

labels_list = ohe.fit_transform(df_test[labels]).toarray()

ohe_labels = ohe.get_feature_names_out(labels)

new_columns = pd.DataFrame(labels_list, columns=ohe_labels)

In [9]:
df_test = df_test.join(new_columns)
df_test.drop(columns=labels, inplace=True)
df_test = df_test.astype('int64')

In [10]:
columnas = ['staff_available', 'Severity of Illness', 'Type of Admission_Emergency', 'Type of Admission_Trauma', 'Type of Admission_Urgent']

df_test.drop(columns=columnas, inplace=True)

In [11]:
df_test

,Age,Department_TB & Chest disease,Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,gender_Female,gender_Male,gender_Other
0,2,0,0,1,0,0,1,0,0
1,5,1,0,0,0,0,1,0,0
2,5,0,0,0,1,0,0,0,1
3,2,0,0,1,0,0,1,0,0
4,5,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
89995,1,0,0,1,0,0,1,0,0
89996,3,0,0,1,0,0,1,0,0
89997,1,0,0,1,0,0,1,0,0
89998,6,0,0,0,1,0,1,0,0


Una vez que tenemos nuestro dataframe listo, podemos aplicar nuestro modelo a los datos para validar. Utilizaremos el modelo del Árbol de Decisión que fue el tuvo mejores resultados.

In [12]:
import joblib

In [16]:
dec_tree = joblib.load('Modelo_Arbol_de_Decision.pkl')

In [17]:
X_test = df_test

In [18]:
pred = dec_tree.predict(X_test)

In [19]:
df_test['pred'] = pred
df_test = df_test['pred']

In [26]:
df_test

0        1
1        1
2        1
3        1
4        1
        ..
89995    1
89996    1
89997    1
89998    1
89999    1
Name: pred, Length: 90000, dtype: int64

In [27]:
df_test.to_csv('ValenteFog.csv', index=False)